# general_demographics-pca.ipynb

### CSc-59867 - Senior Design - Prof. Etemadpour

* Purpose: Present an interactive visualization with the NYC census demographics dataset from NYC Open Data
* Date started: 2021-04-29
* Authors: Xin Chen, Ian S. McBride, Lifu Tao

In [ ]:
from jupyter_dash import JupyterDash

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

When running in JupyterHub or Binder, call the `infer_jupyter_config` function to detect the proxy configuration.

In [ ]:
# JupyterDash.infer_jupyter_proxy_config()

Load and preprocess data

In [ ]:
# df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

Construct the app and callbacks

In [ ]:
app = JupyterDash(__name__)

# Create server variable with Flask server object for use with gunicorn
server = app.server
z
app.layout = html.Div([
    html.Div([
        'Stub content'
    ], style={
    }),
])

In [ ]:
# app.run_server() # Require opening a new browser tab to view app
# app.run_server(mode="inline") # Display app in notebook cell
app.run_server(mode="jupyterlab") # Display app in JupyterLab tab